In [ ]:
# Mail id: kedharnath1992@gmail.com
# Kindly cite if you use the script ""



#############################################
# Import necessary modules
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import math
from math import log
from sklearn.metrics import mean_squared_error

#############################################
# Read the input data and select data for analysis
df = pd.read_excel("data_xgboost.xlsx")
df.head()

pearcol = df.drop(df.columns[[3]], axis ='columns')   # In python the column numbering starts from zero
X = df.iloc[:, 0:4]
Y = df.Stress
list(X.columns)


In [ ]:
# Temperature correction to account for adiabatic heating due to work hardening

# Add isothermal data to ML (XGBoost, SHAP) after the temperature correction

from scipy.integrate import simpson
from numpy import trapz

temp = df.Temperature
sr = df.Strainrate
w = df.Alloying
strain = df.Strain

delT_t = [0] * len(X)
df['New_Temp'] = ''

l = [0] * len(X)
j = 0
for i in range(0,(len(X)-1)):
  if strain[i] > strain[i+1]:
    l[j] = i+1
    j = j + 1

line = [0] * (j+2)
line[1:j+1] = l[0:j]
line[0] = 0
line[j+1] = len(X)

k = 0
for i in range(0,len(line)-1):
  lin = line[i]
  x_lin = strain[lin:line[i+1]]
  y_lin = Y[lin:line[i+1]]
  t = temp[lin:line[i+1]]
  for j in range(0,len(x_lin)):

    Cp = 0.1455 + (0.09544*math.pow(10,-4)*t[k]) + (-68.9/(t[k]*t[k]))
    if w[k] == 0:
      density = 16.6
    if w[k] == 2.5:
      density = 16.67     # https://www.matweb.com/search/datasheet.aspx?matguid=9bf51aa6087c4e578be0240ae02c1e17
    if w[k] == 5:
      density = 16.74
    if w[k] == 10:
      density = 16.9      # https://www.matweb.com/search/GetReference.aspx?matid=100357

    x_trapz = x_lin[0:j+1]
    y_trapz = y_lin[0:j+1]
    integral_trap = trapz(y_trapz, x_trapz)
    delT_t[k] = (0.95*integral_trap) / (density*Cp)   # Check units: Kelvin

    df.New_Temp[k] = t[k] + delT_t[k]
    k = k + 1
df['New_Temp'] = df['New_Temp'].astype(float)
print(df)


In [ ]:
!pip install shap
import shap as shap
import xgboost as xgb

In [ ]:
################################
#         XGBOOST
################################

# Separate training and testing data sets

X1 = df.iloc[:, 1:6]
X = X1.drop(X1.columns[3], axis ='columns')
Y = df.Stress
list(X.columns)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=10)   # 25% data for testing
list(X_test)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', use_rmm = 'True',
                          max_depth = 2,
                          gamma = 1,
                          colsample_bytree = 1,
                          subsample = 0.949495,
                          learning_rate = 0.828283,
                          max_delta_step = 67,
                          random_state = 1,
                          n_estimators = 164,
                          alpha = 6,
                          seed = 22
                          )

xg_reg.fit(X_train,Y_train)
preds = xg_reg.predict(X_test)     # predict y data using X_test
# Low RMSE is desired
rmse_test = np.sqrt(mean_squared_error(Y_test, preds))  # calc RMSE bet y_test & y_predict
print("RMSE: %f" % (rmse_test))

############# Plotting

xgb.plot_importance(xg_reg)
plt.savefig('xgb feature imp.png',dpi = 300)
plt.rcParams['figure.figsize'] = [5,5]

pred_ytrain = xg_reg.predict(X_train)
#train = np.dstack((Y_train, pred_ytrain))
pred_ytest = xg_reg.predict(X_test)

rmse_train = np.sqrt(mean_squared_error(Y_train, pred_ytrain))
print("RMSE: %f" % (rmse_train))

# Pearson linear correlation coefficent
corr_test, _ = pearsonr(Y_test, preds)
print('Pearsons correlation test: %.3f' % corr_test)
corr_train, _ = pearsonr(Y_train, pred_ytrain)
print('Pearsons correlation train: %.3f' % corr_train)

Test_accuracy = xg_reg.score(X_test, Y_test)
print(Test_accuracy)
Train_accuracy = xg_reg.score(X_train, Y_train)
print(Train_accuracy)

############# Plotting
plt.show()
plt.scatter(Y_train, pred_ytrain, c='b', label='Training 75% data', s = 70, alpha=0.5)
plt.scatter(Y_test, pred_ytest, c='r', label='Testing 25% data', s = 70, alpha=0.5)
plt.plot([0, 1600], [0, 1600], color = 'black', linewidth = 3, linestyle='dashed')
plt.xlim(0, 1600)
plt.ylim(0, 1600)

plt.legend(loc="upper left", fontsize = 17, prop={'weight': 'bold'})
plt.xticks(weight = 'bold', fontsize= 10)
plt.yticks(weight = 'bold', fontsize= 10)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)

plt.xlabel('Experimental values (MPa)', fontsize= 20, fontweight='bold')
plt.ylabel('XGBoost predicted values (MPa)', fontsize= 20, fontweight='bold')
plt.savefig('xgb.png',dpi = 300, bbox_inches = "tight")


In [ ]:
################################
#         DEFORMATION MAP
#   Generation of data for LightGBM
################################

pow_srs = np.arange(-6,7,1)
srs = [None] * len(pow_srs)
temp = np.arange(77,1300,100)
w = np.array([0, 2.5, 5, 10])

test_matrix = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})
test_matrix_0 = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})
test_matrix_2 = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})
test_matrix_5 = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})
test_matrix_10 = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})

for t in range(0,len(temp)):
  for sr in range(0,len(pow_srs)):
    srs[sr] = math.pow(10,pow_srs[sr])
    for i in range(0,len(w)):
      if w[i] == 0:
        test_matrix = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'Strain': [0.2], 'New_Temp': [temp[t]]})
        test_matrix_0 = pd.concat([test_matrix_0, test_matrix], ignore_index = True)

      if w[i] == 2.5:
        test_matrix = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'Strain': [0.1], 'New_Temp': [temp[t]]})
        test_matrix_2 = pd.concat([test_matrix_2, test_matrix], ignore_index = True)

      if w[i] == 5:
        test_matrix = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'Strain': [0.1], 'New_Temp': [temp[t]]})
        test_matrix_5 = pd.concat([test_matrix_5, test_matrix], ignore_index = True)

      if w[i] == 10:
        test_matrix = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'Strain': [0.1], 'New_Temp': [temp[t]]})
        test_matrix_10 = pd.concat([test_matrix_10, test_matrix], ignore_index = True)

Stress_0 = xg_reg.predict(test_matrix_0)
Stress_2 = xg_reg.predict(test_matrix_2)
Stress_5 = xg_reg.predict(test_matrix_5)
Stress_10 = xg_reg.predict(test_matrix_10)

test_matrix_0['Stress'] = Stress_0
test_matrix_2['Stress'] = Stress_2
test_matrix_5['Stress'] = Stress_5
test_matrix_10['Stress'] = Stress_10

l = 0
k = 0

m_0 = [None] * ((len(pow_srs) * len(temp)) )
m_2 = [None] * ((len(pow_srs) * len(temp)) )
m_5 = [None] * ((len(pow_srs) * len(temp)) )
m_10 = [None] * ((len(pow_srs) * len(temp)) )

xgb_m = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'New_Temp': [], 'Stress': [], 'm': []})
xgb_m_data = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'New_Temp': [], 'Stress': [], 'm': []})
dm = pd.DataFrame({'x': [], 'y': [], 'z': [], 'z1': []})
dm_0 = pd.DataFrame({'x': [], 'y': [], 'z': [], 'z1': []})
dm_2 = pd.DataFrame({'x': [], 'y': [], 'z': [], 'z1': []})
dm_5 = pd.DataFrame({'x': [], 'y': [], 'z': [], 'z1': []})
dm_10 = pd.DataFrame({'x': [], 'y': [], 'z': [], 'z1': []})

for sr in range(0,len(pow_srs)):
  srs[sr] = math.pow(10,pow_srs[sr])


for t in range(0,len(temp)):
  for sr in range(0,len(pow_srs)-1):
    #srs[sr] = math.pow(10,pow_srs[sr])
    for i in range(0,len(w)):
      if w[i] == 0:
        if sr >= 1:
          m_0[k] = np.log10(Stress_0[l+1]/Stress_0[(l-1)])/np.log10(srs[sr+1]/srs[(sr-1)])
          dm = pd.DataFrame({'x': [temp[t]], 'y': [pow_srs[(sr-1)]], 'z': [Stress_0[l]], 'z1': [m_0[k]]})
          dm_0 = pd.concat([dm_0, dm], ignore_index = True)
          xgb_m = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'New_Temp': [temp[t]], 'Stress': [Stress_0[l]], 'm': [m_0[k]]})
          xgb_m_data = pd.concat([xgb_m_data, xgb_m], ignore_index = True)

      if w[i] == 2.5:
        if sr >= 1:
          m_2[k] = np.log10(Stress_2[l+1]/Stress_2[(l-1)])/np.log10(srs[sr+1]/srs[(sr-1)])
          dm = pd.DataFrame({'x': [temp[t]], 'y': [pow_srs[(sr-1)]], 'z': [Stress_2[l]], 'z1': [m_2[k]]})
          dm_2 = pd.concat([dm_2, dm], ignore_index = True)
          xgb_m = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'New_Temp': [temp[t]], 'Stress': [Stress_0[l]], 'm': [m_0[k]]})
          xgb_m_data = pd.concat([xgb_m_data, xgb_m], ignore_index = True)

      if w[i] == 5:
        if sr >= 1:
          m_5[k] = np.log10(Stress_5[l+1]/Stress_5[(l-1)])/np.log10(srs[sr+1]/srs[(sr-1)])
          dm = pd.DataFrame({'x': [temp[t]], 'y': [pow_srs[(sr-1)]], 'z': [Stress_5[l]], 'z1': [m_5[k]]})
          dm_5 = pd.concat([dm_5, dm], ignore_index = True)
          xgb_m = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'New_Temp': [temp[t]], 'Stress': [Stress_0[l]], 'm': [m_0[k]]})
          xgb_m_data = pd.concat([xgb_m_data, xgb_m], ignore_index = True)

      if w[i] == 10:
        if sr >= 1:
          m_10[k] = np.log10(Stress_10[l+1]/Stress_10[(l-1)])/np.log10(srs[sr+1]/srs[(sr-1)])
          dm = pd.DataFrame({'x': [temp[t]], 'y': [pow_srs[(sr-1)]], 'z': [Stress_10[l]], 'z1': [m_10[k]]})
          dm_10 = pd.concat([dm_10, dm], ignore_index = True)
          xgb_m = pd.DataFrame({'Strainrate': [srs[sr]], 'Alloying': [w[i]], 'New_Temp': [temp[t]], 'Stress': [Stress_0[l]], 'm': [m_0[k]]})
          xgb_m_data = pd.concat([xgb_m_data, xgb_m], ignore_index = True)

    l = l + 1
    k = k + 1




In [ ]:

Z1_0 = dm_0.pivot_table(index='x', columns='y', values='z').T.values
Z1_2 = dm_2.pivot_table(index='x', columns='y', values='z').T.values
Z1_5 = dm_5.pivot_table(index='x', columns='y', values='z').T.values
Z1_10 = dm_10.pivot_table(index='x', columns='y', values='z').T.values

X_unique = np.sort(dm_0.x.unique())
Y_unique = np.sort(dm_0.y.unique())
X, Y = np.meshgrid(X_unique, Y_unique)

#######################################
#       Flow stress at 10% strain
#######################################

plt.figure(figsize=(7,5))
#plt.scatter(X,Y, c=Z_0, cmap='hsv', alpha=0.5)
#plt.contour(X, Y, Z1_0,50, cmap='rainbow', alpha=0.5)
plt.contour(X, Y, Z1_0, 100, cmap='rainbow', alpha=1, levels=np.linspace(0,1400,29))
plt.contourf(X, Y, Z1_0, 100, cmap='rainbow', alpha=0.2, levels=np.linspace(0,1400,29))
plt.colorbar();
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
#plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Temperature (K)', fontsize= 20, fontweight='bold')
plt.ylabel('Log strain rate (/s)', fontsize= 20, fontweight='bold')
#plt.legend(['Pure Ta'], fontsize='x-large', frameon=False)
plt.savefig('stress_Ta.png',dpi = 300, bbox_inches = "tight")
plt.show()

plt.figure(figsize=(7,5))
#plt.scatter(X,Y, c=Z_2, cmap='hsv', alpha=0.5)
#plt.contourf(X, Y, Z1_2, 50, cmap='rainbow', alpha=0.5)
plt.contour(X, Y, Z1_2, 100, cmap='rainbow', alpha=1, levels=np.linspace(0,1400,29))
plt.contourf(X, Y, Z1_2, 100, cmap='rainbow', alpha=0.2, levels=np.linspace(0,1400,29))
plt.colorbar();
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Temperature (K)', fontsize= 20, fontweight='bold')
plt.ylabel('Log strain rate (/s)', fontsize= 20, fontweight='bold')
#plt.legend(['Ta-2.5wt%W'], fontsize='x-large', frameon=False)
plt.savefig('stress_Ta2.5W.png',dpi = 300, bbox_inches = "tight")
plt.show()

plt.figure(figsize=(7,5))
#plt.scatter(X,Y, c=Z_5, cmap='hsv', alpha=0.5)
#plt.contourf(X, Y, Z1_5, 50, cmap='rainbow', alpha=0.5)
plt.contour(X, Y, Z1_5, 100, cmap='rainbow', alpha=1, levels=np.linspace(0,1400,29))
plt.contourf(X, Y, Z1_5, 100, cmap='rainbow', alpha=0.2, levels=np.linspace(0,1400,29))
plt.colorbar();
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Temperature (K)', fontsize= 20, fontweight='bold')
plt.ylabel('Log strain rate (/s)', fontsize= 20, fontweight='bold')
#plt.legend(['Ta-5wt%W'], fontsize='x-large', frameon=False)
plt.savefig('stress_Ta5W.png',dpi = 300, bbox_inches = "tight")
plt.show()

plt.figure(figsize=(7,5))
#plt.scatter(X,Y, c=Z_10, cmap='hsv', alpha=0.5)
plt.contour(X, Y, Z1_10, 100, cmap='rainbow', alpha=1, levels=np.linspace(0,1450,29))
plt.contourf(X, Y, Z1_10, 100, cmap='rainbow', alpha=0.2, levels=np.linspace(0,1400,29))
plt.colorbar();
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('Temperature (K)', fontsize= 20, fontweight='bold')
plt.ylabel('Log strain rate (/s)', fontsize= 20, fontweight='bold')
#plt.legend(['Ta-10wt%W'], fontsize='x-large', frameon=False)
plt.savefig('stress_Ta10W.png',dpi = 300, bbox_inches = "tight")
plt.show()



In [ ]:
################################
# STRESS-STRAIN curve
################################

temp = 77     # in  Kelvin
pow_srs = -3       # in  powers of /s
srs = 1500 #math.pow(10,pow_srs)
w = 10    # 0 to 10 wt% W
strain = np.array([0.002, 0.05, 0.1, 0.2, 0.3])

flow = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})
flow_data = pd.DataFrame({'Strainrate': [], 'Alloying': [], 'Strain': [], 'New_Temp': []})

for i in range(0,len(strain)):
  flow = pd.DataFrame({'Strainrate': [srs], 'Alloying': [w], 'Strain': [strain[i]], 'New_Temp': [temp]})
  flow_data = pd.concat([flow_data, flow], ignore_index = True)

stress_data = xg_reg.predict(flow_data)
plt.scatter(strain,stress_data, alpha=0.5)
plt.ylim(0, 1700)
plt.xlim(0,0.3)
print(stress_data)





In [ ]:
################################
# SHAP analysis
################################

list(X.columns)
# X = X1.drop(X1.columns[[2,3]], axis ='columns')  # SHAP needs same features as XGBoost
shap.initjs()
explainer = shap.Explainer(xg_reg, X) # (xg_reg, X_test), (xg_reg, X_train), & (xg_reg) shows DIFFERENCE
shap_values = explainer(X)   # if X_test is changed to X_train then shows DIFFERENCE; but mostly X_test is used https://www.kaggle.com/code/dansbecker/shap-values
feature_names = [ a + ": " + str(b) for a,b in zip(X.columns, np.abs(shap_values.values).mean(0))]
shap.summary_plot(shap_values, plot_type='violin', feature_names = feature_names, color_bar = False, show = False, alpha=0.5)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=15)
cbar.ax.tick_params(direction='out', length=6, width=2,  grid_alpha=0.5)

plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.xlabel('SHAP value', fontsize= 15, fontweight='bold')

plt.savefig("shap violin.png", dpi = 300, bbox_inches = "tight")
plt.close()

shap.summary_plot(shap_values, plot_type='violin', feature_names=feature_names)

shap.plots.scatter(shap_values[:,"New_Temp"], color=shap_values[:,"Strainrate"], dot_size=80, cmap='rainbow', alpha=0.5)
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.savefig("shap T correlation.png", dpi = 300, bbox_inches = "tight")


shap.plots.scatter(shap_values[:,"Alloying"], color=shap_values[:,"Strainrate"], dot_size=80, cmap='rainbow', alpha=0.5)
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.savefig("shap W correlation.png", dpi = 300, bbox_inches = "tight")


shap.plots.scatter(shap_values[:,"Strainrate"], color=shap_values[:,"Alloying"], dot_size=80, cmap='rainbow', alpha=0.5)
plt.xticks(weight = 'bold', fontsize= 13)
plt.yticks(weight = 'bold', fontsize= 13)
plt.rcParams["axes.linewidth"] = 1.5
plt.tick_params(direction='out', length=6, width=2, grid_alpha=0.5)
plt.savefig("shap SRS correlation.png", dpi = 300, bbox_inches = "tight")

